In [1]:
import numpy as np
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import gensim
file_name = 'ALICE.txt'
data=open(file_name)
corona_data = [text for text in data if text.count(' ') >= 2]
vectorize = Tokenizer()
vectorize.fit_on_texts(corona_data)
corona_data = vectorize.texts_to_sequences(corona_data)
total_vocab = sum(len(s) for s in corona_data)
word_count = len(vectorize.word_index) + 1
window_size = 2

In [68]:
my_dict = vectorize.word_index

vocab = []
for key, value in my_dict.items():
    vocab.append(key)

my_dict

{'the': 1,
 "'": 2,
 'and': 3,
 'to': 4,
 'a': 5,
 'she': 6,
 'of': 7,
 'it': 8,
 'said': 9,
 'alice': 10,
 'in': 11,
 'was': 12,
 'you': 13,
 'i': 14,
 'that': 15,
 'as': 16,
 'her': 17,
 'at': 18,
 'on': 19,
 'with': 20,
 'all': 21,
 'had': 22,
 'be': 23,
 'for': 24,
 'so': 25,
 'very': 26,
 'not': 27,
 'but': 28,
 'this': 29,
 'little': 30,
 "'i": 31,
 'they': 32,
 'out': 33,
 'he': 34,
 'is': 35,
 'down': 36,
 'what': 37,
 'up': 38,
 'one': 39,
 'his': 40,
 'about': 41,
 'them': 42,
 'like': 43,
 'were': 44,
 'herself': 45,
 'know': 46,
 'then': 47,
 'went': 48,
 'again': 49,
 'would': 50,
 'no': 51,
 'have': 52,
 'could': 53,
 'if': 54,
 'thought': 55,
 'when': 56,
 'do': 57,
 'or': 58,
 'there': 59,
 'time': 60,
 'into': 61,
 'see': 62,
 'me': 63,
 'queen': 64,
 'off': 65,
 'did': 66,
 'your': 67,
 'king': 68,
 'began': 69,
 'by': 70,
 'its': 71,
 "'and": 72,
 'way': 73,
 'an': 74,
 'mock': 75,
 'quite': 76,
 'my': 77,
 'who': 78,
 "don't": 79,
 'hatter': 80,
 'turtle': 81,
 'gry

In [2]:
from cbow_model import *

In [3]:
# cbow_model = cbow_model(vocab_size = total_vocab, embedding_dim = 100 , window_size = window_size)
# # Create the training data
# X_cbow, y_cbow = generate_data_cbow(corona_data, window_size, total_vocab)
# X_cbow.shape, y_cbow.shape
# history = cbow_model.fit(X_cbow, y_cbow, batch_size=64, epochs=150, verbose=2)
# cbow_model.save('cbow_model.h5')

In [4]:
from keras.models import load_model

In [53]:
cbow_model = load_model('cbow_model.h5')
cbow_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            2757400   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 27574)             2784974   
                                                                 
Total params: 5,542,374
Trainable params: 5,542,374
Non-trainable params: 0
_________________________________________________________________


In [6]:
dimensions=100
vect_file = open('vectors.bin' ,'w')
vect_file.write('{} {}\n'.format(total_vocab,dimensions))

10

In [7]:
weights = cbow_model.get_weights()[0]
for text, i in vectorize.word_index.items():
    final_vec = ' '.join(map(str, list(weights[i, :])))
    vect_file.write('{} {}\n'.format(text, final_vec))
vect_file.close()

In [8]:
import numpy as np

def load_vectors(fname):
    fin = open(fname, 'r')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.strip().split()
        word = tokens[0]
        vec = np.array(tokens[1:], dtype=np.float32)
        data[word] = vec
    fin.close()
    return data

vectors = load_vectors('vectors.txt')


In [9]:
def predict_vector(text, model, vectors):
    # Convert the input text to a list of word vectors
    words = text.split()
    vecs = []
    for word in words:
        if word in vectors:
            vecs.append(vectors[word])
    # Calculate the average vector for the input text
    if vecs:
        avg_vec = sum(vecs) / len(vecs)
    else:
        avg_vec = np.zeros((model.vector_size,), dtype=np.float32)
    # Predict the output vector using the trained CBOW model
    input_vec = avg_vec.reshape(1, -1)
    output_vec = model.predict(input_vec)
    return output_vec


In [10]:
vocab_size = len(vectors)
embedding_size = len(list(vectors.values())[0])

In [11]:
# example text input
text = "the red blind dog hit his we head".split()


In [12]:
# Convert the context to a vector
context_vector = np.zeros((len(text),))
for word in text:
    if word in text:
        context_vector[text.index(word)] += 1

# Normalize the context vector
context_vector /= np.linalg.norm(context_vector)

# Reshape the context vector to have a shape of (1, len(vocabulary))
context_vector = context_vector.reshape(1, len(text))

In [13]:
context_vector

array([[0.35355339, 0.35355339, 0.35355339, 0.35355339, 0.35355339,
        0.35355339, 0.35355339, 0.35355339]])

In [74]:
# Define the vocabulary and word-to-id dictionary
vocabulary = vocab
word_to_id = {word: i for i, word in enumerate(vocabulary)}

# Define the id-to-word dictionary
id_to_word = {i: word for word, i in my_dict.items()}


In [105]:
# Define the id-to-word dictionary
id_to_word = {i: word for word, i in word_to_id.items()}
for i in range(len(vocabulary)):
    id_to_word.setdefault(i, '<unknown>')

# Define the input sentence
sentence = " we think VERY much of ".lower()
words = sentence.split()

# Pad or truncate the sentence to length 4
if len(words) < 4:
    words += [''] * (4 - len(words))
else:
    words = words[:4]

# Convert the words to integer-encoded IDs
word_ids = [word_to_id[word] for word in words]

# Predict the target word
x = np.array([word_ids])
y_pred = cbow_model.predict(x)
target_word_id = np.argmax(y_pred)
target_word_id


1/1 [==============================] - 1s 618ms/step


13

In [106]:
target_word = id_to_word[target_word_id]
print("Predicted target word:", target_word)

Predicted target word: i


In [32]:
# Predict the target word
prediction = cbow_model.predict(context_vector)

# Get the index of the predicted word
predicted_index = np.argmax(prediction)

# Get the predicted word from the vocabulary
predicted_word = text[predicted_index]

print("The predicted next word is:", predicted_word)

ValueError: in user code:

    File "c:\Users\basel\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\basel\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\basel\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\basel\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "c:\Users\basel\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users